# Delete duplicates events

In .env

```
EXTRACTOR_SECRET_KEY=
EXTRACTOR_URL=
MONGODB_URL=
```

Later on, in ORGS_TO_PROCESS, put an org id in it. 

In [1]:
# Set current working dir
import os
from dotenv import load_dotenv

# Path with .env
os.chdir("/Users/nicolasoulianov/phospho/phospho/backend/scripts")

load_dotenv()

True

In [3]:
import pymongo

# Initialize pymongo client
client = pymongo.MongoClient(os.getenv('MONGODB_URL'))
db = client[os.getenv('MONGODB_NAME')]

In [6]:
db.name

'production'

In [25]:
ORGS_TO_PROCESS = [
#  "select an org"    
]

In [158]:
import datetime 

# Get end date
end_date = datetime.datetime.now() 
# end_date = datetime.datetime.now() - datetime.timedelta(weeks=1)

end_date_ts = int(end_date.timestamp())

# Get the start date
start_date = end_date - datetime.timedelta(days=10)#datetime.timedelta(weeks=1)
start_date_ts = int(start_date.timestamp())

In [159]:
# count nb events
db["events"].count_documents({})

116559

In [160]:
# Fetch duplicate events id
duplicated_events = list(
    db["events"].aggregate(
        [
            {
                "$match": {
                    # "org_id": {"$in": ORGS_TO_PROCESS},
                    "created_at": {"$gte": start_date_ts, "$lte": end_date_ts},
                }
            },
            {
                # from earliest to latest
                "$sort": {"created_at":  1}
            },
            {
                "$group": {
                    "_id": {"task_id": "$task_id", "event_name": "$event_name"},
                    "dups": {"$push": "$_id"},
                    "count": {"$sum": 1},
                },
            },
            {"$match": {"count": {"$gt": 1}}},
        ],
    )
)

len(duplicated_events)

1488

In [161]:
sum(len(k["dups"]) for k in duplicated_events)

14803

In [162]:
from tqdm import tqdm

# Drop duplicated events based on _id
events_to_delete = []
for duplicated_event in tqdm(duplicated_events):
    # Get the first event
    first_event = duplicated_event["dups"][0]
    # Get the rest of the events
    rest_of_events = duplicated_event["dups"][1:]

    if len(rest_of_events) == 0:
        continue

    events_to_delete.extend(rest_of_events)

# Delete the rest of the events
# print(f"Deleting {len(rest_of_events)} events")
print(f"Deleting {len(events_to_delete)} events")

db["events"].delete_many({"_id": {"$in": events_to_delete}})

100%|██████████| 1488/1488 [00:00<00:00, 1307589.43it/s]


Deleting 13315 events


DeleteResult({'n': 13315, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1710192546, 3615), 'signature': {'hash': b'x\xd9\xdb\xb8B\xdcC\x99\xc8\xb6\x8f\xfdDM\x95\x87\xab\xaa\x13\xc2', 'keyId': 7314743902576050178}}, 'operationTime': Timestamp(1710192546, 3611)}, acknowledged=True)